# HW 1

### Decision Variables
Let $x_{i j}$ be the number of units of product $i$ to be produced in week $j$, where $i=$ $1,2, \ldots, 100$ and $j=1,2, \ldots, 52$.

### Objective Function
Maximize:
Profit - Holding Cost $=\sum_{i=1}^{100} \sum_{j=1}^{52} x_{i j} \times$ profit $_i-\sum_{i=1}^{500} \sum_{j=1}^{52}$ inventory $_{i j} \times$ holding $_j$

Constraints
1. $x_{i j} \leq \operatorname{demand}_{i j}$ (cannot produce more than demand)
2. inventory $_{i j} \geq 0$ (inventory cannot be negative)
3. Material availability constraint:
$$
\text { inventory }_{i j}+\sum_{k=1}^{100} x_{k j} \times \text { requirements }_{i k} \leq \text { availability }_{i j}
$$

In [1]:
using Pkg
Pkg.add("CSV")
Pkg.add("DataFrames")
Pkg.add("JuMP")
Pkg.add("Gurobi")

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...


  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`


   Resolving package versions...


  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`


   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`


   Resolving package versions...


  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`


In [2]:
using CSV, DataFrames, JuMP, Gurobi

In [3]:
# Load data
availability = CSV.read("data/availability.csv", DataFrame) |> Matrix
demand = CSV.read("data/demand.csv", DataFrame) |> Matrix
holding = CSV.read("data/holding.csv", DataFrame) |> Vector
profit = CSV.read("data/profit.csv", DataFrame) |> Vector
requirements = CSV.read("data/requirements.csv", DataFrame) |> Matrix


num_products = 100
num_weeks = 52
num_materials = 500

# Initialize the model
model = Model(Gurobi.Optimizer)

# Decision variables
@variable(model, x[1:num_products, 1:num_weeks] >= 0)
@variable(model, inventory[1:num_materials, 1:num_weeks] >= 0)

# Objective function
@objective(model, Max, sum(x[i, j] * profit[i] for i in 1:num_products, j in 1:num_weeks) -
                      sum(inventory[i, j] * holding[j] for i in 1:num_materials, j in 1:num_weeks))

# Demand constraints
for i in 1:num_products
    for j in 1:num_weeks
        @constraint(model, x[i, j] <= demand[i, j])
    end
end

# Material availability constraints
for i in 1:num_materials
    for j in 1:num_weeks
        @constraint(model, inventory[i, j] + sum(x[k, j] * requirements[i, k] for k in 1:num_products) <= availability[i, j])
    end
end

# Solve the model
optimize!(model)

# Extract results
total_profit = objective_value(model)
x_values = value.(x)
inventory_values = value.(inventory)

# Calculate total holding cost
total_holding_cost = sum(inventory_values[i, j] * holding[j] for i in 1:num_materials, j in 1:num_weeks)

# Print results
println("Total Profit: ", total_profit)
println("Total Holding Cost: ", total_holding_cost)


MethodError: MethodError: no method matching (Matrix)(::CSV.File)

Closest candidates are:
  (Array{T, N} where T)(::AbstractArray{S, N}) where {S, N}
   @ Core boot.jl:498
  (Matrix)(!Matched::Union{LinearAlgebra.QR, LinearAlgebra.QRCompactWY})
   @ LinearAlgebra /Applications/Julia-1.9.app/Contents/Resources/julia/share/julia/stdlib/v1.9/LinearAlgebra/src/qr.jl:449
  (Matrix)(!Matched::LinearAlgebra.Eigen)
   @ LinearAlgebra /Applications/Julia-1.9.app/Contents/Resources/julia/share/julia/stdlib/v1.9/LinearAlgebra/src/eigen.jl:654
  ...
